In [1]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import pickle
import time as time
import math, cmath
import auxiliary_library as me

from IPython.display import display, Math, Latex

In [68]:
def H_ij_matrix(size, chain_type, Hamiltonian_parameters, basis):
    
    one_body_biglist = me.one_body_spin_ops(N)
    H_H = me.Heisenberg_Hamiltonian(one_body_biglist, chain_type, size, False, 
                                    Hamiltonian_parameters[0], Hamiltonian_parameters[1], Hamiltonian_parameters[2], 
                                    Hamiltonian_parameters[3])
    
    basis = []
    for i in range(len(one_body_biglist)):
        for j in range(len(one_body_biglist[i])):
            basis.append(one_body_biglist[i][j])
    
    coeffs_list = [me.mod_HS_inner_prod(op1, commutator(H_H, op2)) for op1 in basis for op2 in basis]
        
    # convert list to numpy array
    coeffs_matrix = np.asarray(coeffs_list)
    # reshape array into 4 rows x 2 columns, and transpose the result
    coeffs_matrix = coeffs_matrix.reshape(len(basis), len(basis))
    
    return coeffs_matrix
    

In [83]:
N = 2
Hparameters = [.15, .25, .1, 1.]
one_body_biglist = me.one_body_spin_ops(N)
hij = H_ij_matrix(N, "XYZ", Hparameters, one_body_biglist)

array([[0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+1.57079633j, 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+1.57079633j, 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.-1.57079633j, 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.-1.57079633j,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.

In [104]:
def dPhidt(dmatrix):
    
    vector = np.full((8,1),1)

    Lmu = []

    for i in range(len(dmatrix)):
        Lmu.append(sum(dmatrix[i][b] * vector[b] for b in range(len(hij[0]))))
    
    return Lmu

dPhidt(hij)


[array([0.+0.j]),
 array([0.+0.j]),
 array([0.+1.57079633j]),
 array([0.+1.57079633j]),
 array([0.-1.57079633j]),
 array([0.-1.57079633j]),
 array([0.+0.j]),
 array([0.+0.j])]

In [106]:
from scipy.integrate import odeint
from scipy.integrate import solve_ivp

vector1 = (0,0,1,-1,-1,0,0)

x = np.linspace(0, 1, 100)
sol = odeint(dPhidt(hij), y0 = vector1, t = x, tfirst = True)

error: The function and its Jacobian must be callable functions.

In [72]:
N = 4
Hparameters = [.15, .25, .1, 1.]a
one_body_biglist = me.one_body_spin_ops(N)
hij = H_ij_matrix(N, "XYZ", Hparameters, one_body_biglist)
len(hij)

16

In [53]:
N = 2 
Jx = .15; Jy = .25; Jz = .1; h = 1.

one_body_biglist = me.one_body_spin_ops(N)
two_body_biglist = me.two_body_spin_ops(one_body_biglist, N, True)
H_H = me.Heisenberg_Hamiltonian(one_body_biglist, "XYZ", N, False, Jx, Jy, Jz, h)

def commutator(A, B):
    result = 0
    #if A.dims[0][0] == B.dims[0][0]: 
    #    pass
    #else:
    #    raise Exception("Incompatible Qobj dimensions")
    result = A*B-B*A

    return result

one_body_biglist[0][0]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

In [61]:
len(basis)

8

In [67]:
basis = []
for i in range(len(one_body_biglist)):
    for j in range(len(one_body_biglist[i])):
        basis.append(one_body_biglist[i][j])

coeffs_list = [me.mod_HS_inner_prod(op1, commutator(H_H, op2)) for op1 in basis for op2 in basis]
coeffs_list

# convert list to numpy array
coeffs_matrix = np.asarray(coeffs_list)
# reshape array into 4 rows x 2 columns, and transpose the result
coeffs_matrix = coeffs_matrix.reshape(8, 8).T 
coeffs_matrix * np.full((8,8), 2)

array([[0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.-3.14159265j, 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.-3.14159265j, 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+3.14159265j, 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+3.14159265j,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ,
        0.+0.j        , 0.+0.j        , 0.+0.j        , 0.

In [32]:
N = 2
coeff_matrix = [("<O",i, "[H, Q", j,"]>") for i in range(N) for j in range(N)]
coeffs_matrix = [i*j for i in range(N) for j in range(N)]

In [40]:
data = np.array( [0, 2, 7, 6] )
shape = ( 2, 2 )
data.reshape( shape )
data


array([0, 2, 7, 6])

In [27]:
np.array(mat)

array([[   2,   10,   20],
       [ 200, 4000,  500]])